In [1]:
from dataclasses import dataclass, field
import open3d as o3d
import numpy as np
import time
from pathlib import Path
from bbox import  BBox3D
from scipy.spatial.transform import Rotation as R

# # The detected car is being overtaken.
# yaw_filter=[[0, 30], [-0, -30]]
# pcd_track_uuids = np.load("../../data/P04/raw_data/000/000003/pcd.npy",  allow_pickle=True).item()
# instance = pcd_track_uuids['293fe4b1-c557-4d2f-a214-9308a50f153b']

# # A car is driving in the opposite direction and is turning.
# yaw_filter=[[150, 180], [-90, -180]]
# pcd_track_uuids = np.load("../../data/P04/raw_data/000/000009/pcd.npy",  allow_pickle=True).item()
# instance = pcd_track_uuids['a02314d7-191c-496f-9b01-2dec0602e765']

# # The car drive in a perpendicular direction.
# yaw_filter=[[45, 135], [-0, -0]]
# pcd_track_uuids = np.load("../../data/P04/raw_data/000/000011/pcd.npy",  allow_pickle=True).item()
# instance = pcd_track_uuids['073a72b3-1750-4897-9c2c-0944a0d4ac89']

# # a car is turning right.
# yaw_filter=[[0, 135], [-0, -20]]
# pcd_track_uuids = np.load("../../data/P04/raw_data/000/000021/pcd.npy",  allow_pickle=True).item()
# instance = pcd_track_uuids['ffdcdedb-067d-479c-92c4-72c4ba7c993a']

# # The car drive in a perpendicular direction.
# yaw_filter=[[45, 180], [-0, -0]]
# pcd_track_uuids = np.load("../../data/P04/raw_data/000/000030/pcd.npy",  allow_pickle=True).item()
# instance = pcd_track_uuids['2383ced0-16f8-4732-aede-142a8f4c6b04']

# # The ego-car is being overtaken
# yaw_filter=[[0, 20], [-0, -20]]
# pcd_track_uuids = np.load("../../data/P04/raw_data/000/000049/pcd.npy",  allow_pickle=True).item()
# instance = pcd_track_uuids['2d1e69f6-9aa9-4b89-a700-c51d12ec112c']

# # The ego-car is turning left
# yaw_filter=[[0, 120], [-0, -45]]
# pcd_track_uuids = np.load("../../data/P04/raw_data/001/001000/pcd.npy",  allow_pickle=True).item()
# instance = pcd_track_uuids['66f6a5e9-c37a-45e6-b3f8-00013daccb56']

# # The detected-car is turning right and then goes straight
# yaw_filter=[[0, 120], [-0, -45]]
# pcd_track_uuids = np.load("../../data/P04/raw_data/001/001005/pcd.npy",  allow_pickle=True).item()
# instance = pcd_track_uuids['be464e86-89ee-40d7-a417-2de73cac0a42']

# # The detected-car comes from an alley and merge into main road.
# yaw_filter=[[150, 180], [-120, -180]]
# pcd_track_uuids = np.load("../../data/P04/raw_data/001/001006/pcd.npy",  allow_pickle=True).item()
# instance = pcd_track_uuids['428c8029-4f08-4182-ae5e-92581d303779']

# # The detected-car goes straight and turning right.
# yaw_filter=[[0, 45], [-0, -100]]
# pcd_track_uuids = np.load("../../data/P04/raw_data/001/001007/pcd.npy",  allow_pickle=True).item()
# instance = pcd_track_uuids['051af1a0-3b0c-41bc-88e7-10ada6eb10cf']

# # The detected-car comes from opposite direection and turns left.
# yaw_filter=[[170, 180], [-90, -180]]
# pcd_track_uuids = np.load("../../data/P04/raw_data/001/001027/pcd.npy",  allow_pickle=True).item()
# instance = pcd_track_uuids['b0dd7489-5721-4202-a700-3364adf5e461']

# # The detected-car is turning right
# yaw_filter=[[0, 120], [-0, -30]]
# pcd_track_uuids = np.load("../../data/P04/raw_data/001/001035/pcd.npy",  allow_pickle=True).item()
# instance = pcd_track_uuids['5ed56a87-e8c9-4e06-bc9a-ceb4a8a44360']

# # The detected-car is turning left
# yaw_filter=[[0, 45], [-0, -45]]
# pcd_track_uuids = np.load("../../data/P04/raw_data/001/001038/pcd.npy",  allow_pickle=True).item()
# instance = pcd_track_uuids['1d33dc72-e987-4140-9a7f-b653b2d3b41d']

# # the car drive in perpendicular direction
# yaw_filter=[[40, 180], [-0, -0]]
# pcd_track_uuids = np.load("../../data/P04/raw_data/001/001039/pcd.npy",  allow_pickle=True).item()
# instance = pcd_track_uuids['ff60da28-a51a-4fdc-aceb-fd9605addaa0']

# the car u turns -
yaw_filter=[[0, 180], [-0, -50]]
pcd_track_uuids = np.load("../../data/P04/raw_data/001/001046/pcd.npy",  allow_pickle=True).item()
instance = pcd_track_uuids['945feec5-d2f5-49da-ab6d-c71f9402a23f']


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
@dataclass
class BoundingBox3D:
    '''
    pose in s-frame
    '''
    x: float
    y: float
    z: float
    length: float
    width: float
    height: float
    rot: float
    iou: BBox3D = field(init=False, repr=False)

    def __post_init__(self):
        r = R.from_matrix(self.rot)
        q8d_xyzw = r.as_quat()
        q8d = np.array([q8d_xyzw[3], q8d_xyzw[0], q8d_xyzw[1], q8d_xyzw[2]])

        self.iou: BBox3D = BBox3D(self.x, self.y, self.z, 
                                         self.length, self.width, 
                                         self.height, q=q8d)

In [3]:
def translate_boxes_to_open3d_instance(bbox, crop=False):
    """
          4 -------- 6
         /|         /|
        5 -------- 3 .
        | |        | |
        . 7 -------- 1
        |/         |/
        2 -------- 0
    https://github.com/open-mmlab/OpenPCDet/blob/master/tools/visual_utils/open3d_vis_utils.py
    """
    center = [bbox.x, bbox.y, bbox.z]
    lwh = [bbox.length, bbox.width, bbox.height]
    if not crop:
        box3d = o3d.geometry.OrientedBoundingBox(center, bbox.rot, lwh)
    else:
        lwh = [bbox.length, bbox.width, bbox.height * 0.9]
        box3d = o3d.geometry.OrientedBoundingBox(center, bbox.rot, lwh)

    line_set = o3d.geometry.LineSet.create_from_oriented_bounding_box(box3d)
    lines = np.asarray(line_set.lines)
    lines = np.concatenate([lines, np.array([[1, 4], [7, 6]])], axis=0)

    line_set.lines = o3d.utility.Vector2iVector(lines)
    

    return line_set, box3d

In [4]:
def change_bbox(line_set, bbox):
    center = [bbox.x, bbox.y, bbox.z]
    lwh = [bbox.length, bbox.width, bbox.height]
    box3d = o3d.geometry.OrientedBoundingBox(center, bbox.rot, lwh)
        
    lines = np.asarray(line_set.lines)
    lines = np.concatenate([lines, np.array([[1, 4], [7, 6]])], axis=0)

    line_set.lines = o3d.utility.Vector2iVector(lines)

In [5]:
def get_bbox(pcd):
    x = pcd['bbox'][0]
    y = pcd['bbox'][1]
    z = pcd['bbox'][2]
    l = pcd['bbox'][4]
    w = pcd['bbox'][3]
    h = pcd['bbox'][5]
    rot = pcd['T_cam_obj'][:3, :3]
    bbox = BoundingBox3D(x,y,z,l,w,h,rot)
    return bbox

In [6]:
def clean_data(mtx, yaw_filter=[[0, 180], [-0, -180]]):
    r = R.from_matrix(mtx[:3, :3])
    euler = r.as_euler('zxy', degrees=True)[0]
    print("euler", euler)
    if euler < 0:
        if euler > yaw_filter[1][0] or euler < yaw_filter[1][1]:
            print("before minus", euler)
            mtx[0][0] = -mtx[0][0] 
            mtx[0][1] = -mtx[0][1] 
            mtx[1][0] = -mtx[1][0] 
            mtx[1][1] = -mtx[1][1]
            print("after minus", euler)
            return mtx
    else:
        if euler < yaw_filter[0][0] or euler > yaw_filter[0][1]:
            print("before plus", euler)
            mtx[0][0] = -mtx[0][0] 
            mtx[0][1] = -mtx[0][1] 
            mtx[1][0] = -mtx[1][0] 
            mtx[1][1] = -mtx[1][1]
            print("after plus", euler)
            return mtx
    return mtx

In [7]:
# seq = Path("001/001038")

# result_dir = Path("/home/ohmpr/master_bonn/Modules/3rd_semester/P02/sandbox/Deep_SDF_Debug/data/P04_pcd") / seq

# if not result_dir.exists():
#     print("test")
#     result_dir.mkdir(parents=True, exist_ok=True)

instance_id = 0
max_frame = 0
for k, _ in instance.items():
    frame_number = len(instance[k])
    if frame_number > max_frame:
        max_frame = frame_number
        first_instance = k
points = instance[first_instance]
# Find the first frame
first_frame = 0
for k, v in points.items():
    first_frame = k
    break

# Visualize results
vis = o3d.visualization.Visualizer()
vis.create_window()
# Coordinate frame
axis_pcd = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0, 0, 0])
vis.add_geometry(axis_pcd)

pts_str = 'pts_cam' # dict_keys(['T_cam_obj', 'pts_cam', 'surface_points', 'bbox'])

# Extracted point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[first_frame][pts_str])
vis.add_geometry(pcd)
# o3d.io.write_point_cloud(f"{result_dir}/{first_frame}.pcd", pcd)

# Orientation
mtx = points[first_frame]['T_cam_obj']
mtx = clean_data(mtx, yaw_filter)
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=3, origin=[0, 0, 0])
coordinate_frame.transform(mtx)
prev_mtx = mtx
vis.add_geometry(coordinate_frame)

# Detected bounding box 
bbox = get_bbox(points[first_frame])
line_set, box3d = translate_boxes_to_open3d_instance(bbox)
vis.add_geometry(line_set)


for frame_id, points_scan in points.items():
    if frame_id == first_frame:
        pass

    # Point Cloud
    pcd.points = o3d.utility.Vector3dVector(points_scan[pts_str])
    # o3d.io.write_point_cloud(f"{result_dir}/{frame_id}.pcd", pcd)
    vis.update_geometry(pcd)
    
    # Orientation
    coordinate_frame.transform(np.linalg.inv(prev_mtx)) # undo previous transformation
    mtx = points_scan['T_cam_obj']
    mtx = clean_data(mtx, yaw_filter)
    coordinate_frame.transform(mtx)
    vis.update_geometry(coordinate_frame)
    
    # Detected bounding box 
    line_set.transform(np.linalg.inv(prev_mtx)) # undo previous transformation
    bbox = get_bbox(points_scan)
    change_bbox(line_set, bbox)
    line_set.transform(mtx)
    line_set.paint_uniform_color(np.array([60. / 255., 180. / 255., 75. / 255.])) # GREEN
    vis.update_geometry(line_set)





    prev_mtx = mtx
    vis.poll_events()
    vis.update_renderer()
    time.sleep(0.1)

vis.destroy_window()

euler -1.530006991953983
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
euler -1.530006991953983
euler -179.3992798478115
before minus -179.3992798478115
after minus -179.3992798478115
euler 177.9625595852711
euler -6.4027732288678
euler 4.324366711717387
euler -2.8964273165500543
euler 3.295685581200106
euler -0.2520512774879171
euler 177.91845022361116
euler 6.182888505634535
euler -0.12208643796091447
euler -0.3679732480813735
euler 6.658105747327344
euler 9.556155012163751
euler -169.97365977432892
before minus -169.97365977432892
after minus -169.97365977432892
euler 177.74060575800237
euler 7.00510667792067
euler -162.80895525952562
before minus -162.80895525952562
after minus -162.80895525952562
euler -163.6982595498396
before minus -163.6982595498396
after minus -163.6982595498396
euler 21.620536709210718
euler 28.894141760140695
euler 28.260737337270392
euler 31.02275655459683
euler 40.29105005073734
euler 44.74121882332412
euler 47.131081750715175
eul